In [ ]:
import numpy as np
import pandas as pd

# Import and Take a look at the data

In [ ]:
input_dir = "../input/tabular-playground-series-jun-2021/"
train_df = pd.read_csv(input_dir + "train.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.nunique().sort_values(ascending=False)[:20]

# Preprocessing for modeling

get features and transform target

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(train_df["target"])

X = train_df.iloc[:, 1:-1].values

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_idf = TfidfTransformer()
X = tf_idf.fit_transform(X)

stratified split the data into training set and validation set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y,
                                                      test_size=0.1, random_state=42)

# Train a Random Forest

train a random forest without further tuning

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, max_depth=15, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
def log_loss(pred_probs, y_valid):
    pred_probs = pred_probs + 1e-15 # numerical stability
    y_valid = y_valid.reshape(-1, 1) 
    
    # get probs for each class
    preds_class = np.take_along_axis(pred_probs, y_valid, axis=1)
    return -1 * np.log(preds_class).mean()

Performance on validation set

In [ ]:
preds = rf.predict_proba(X_valid)
log_loss(preds, y_valid)

# Make predictions and submit

In [ ]:
test_df = pd.read_csv(input_dir + "test.csv")
X_test = tf_idf.transform(test_df.iloc[:, 1:].values)
test_preds = rf.predict_proba(X_test)

In [ ]:
sub = pd.read_csv(input_dir + "sample_submission.csv")
sub.iloc[:, 1:] = test_preds
sub.to_csv("submission.csv", index=False)